In [ ]:
import glob
import pandas as pd
pd.set_option('display.max_colwidth', -1)  
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['animation.html'] = 'jshtml'
import seaborn as sns

import tensorflow as tf
import tensorflow.keras as keras

import pydicom

In [ ]:
data_dir = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/'

df_train = pd.read_csv(data_dir+'train_labels.csv')
EXCLUDE = [109, 123, 709]
df_train = df_train[~df_train.BraTS21ID.isin(EXCLUDE)]

In [ ]:
df_train

One image reading 

In [ ]:
data = pydicom.dcmread('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T1w/Image-24.dcm')
data

Making all BraTS21ID of same length by adding leading zero if required.


In [ ]:
def make_same_length(patient_id):
    zeros = 5 - len(str(patient_id))
    if zeros > 0:
        prefix = ''.join(['0' for i in range(zeros)])
    else:
        return patient_id
    return prefix+str(patient_id)

In [ ]:
# Add the full paths for each id for different types of sequences to the csv 


df_train['BraTS21ID'] = df_train['BraTS21ID'].apply(make_same_length)

# Add all the paths to the  for easy access
df_train['flair'] = df_train['BraTS21ID'].apply(lambda file_id : data_dir+'train/'+file_id+'/FLAIR/')
df_train['t1w'] = df_train['BraTS21ID'].apply(lambda file_id : data_dir+'train/'+file_id+'/T1w/')
df_train['t1wce'] = df_train['BraTS21ID'].apply(lambda file_id : data_dir+'train/'+file_id+'/T1wCE/')
df_train['t2w'] = df_train['BraTS21ID'].apply(lambda file_id : data_dir+'train/'+file_id+'/T2w/')
df_train.head()

In [ ]:
df_test = pd.read_csv(data_dir+'sample_submission.csv')

df_test['BraTS21ID'] = df_test['BraTS21ID'].apply(make_same_length)

# Add all the paths to the df for easy access
df_test['flair'] = df_test['BraTS21ID'].apply(lambda file_id : data_dir+'test/'+file_id+'/FLAIR/')
df_test['t1w'] = df_test['BraTS21ID'].apply(lambda file_id : data_dir+'test/'+file_id+'/T1w/')
df_test['t1wce'] = df_test['BraTS21ID'].apply(lambda file_id : data_dir+'test/'+file_id+'/T1wCE/')
df_test['t2w'] = df_test['BraTS21ID'].apply(lambda file_id : data_dir+'test/'+file_id+'/T2w/')


In [ ]:
df_test

In [ ]:

import os
import glob
from tqdm import tqdm_notebook as tqdm
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import transforms, utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import tensorflow as tf

In [ ]:

def get_image(path, voi_lut=True, fix_monochrome=True,img_size=256):
    '''
    Returns the image data as a numpy array.
    ''' 
    data = pydicom.dcmread(path)
    if voi_lut:
        img = apply_voi_lut(data.pixel_array, data)
    else:
        img = data.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and data.PhotometricInterpretation == "MONOCHROME1":
        img = np.amax(img) - img
    img = img - np.min(img)
    img = img / np.max(img)
    img = (img * 255).astype(np.uint8)
    img = cv2.resize(img, (img_size, img_size))
        
    return img

https://www.kaggle.com/xuxu1234/efficientnet3d-for-mri

In [ ]:
def load_3d_dicom_images(row):
    """
    we will use some heuristics to choose the slices to avoid any numpy zero matrix (if possible)
    """
    flair = sorted(glob.glob(f"{row['flair']}/*.dcm"))
    t1w = sorted(glob.glob(f"{row['t1w']}/*.dcm"))
    t1wce = sorted(glob.glob(f"{row['t1wce']}/*.dcm"))
    t2w = sorted(glob.glob(f"{row['t2w']}/*.dcm"))
    img_size=256
    
    ''' flair_img = np.array([get_image(a) for a in flair[len(flair)//2 - 10:len(flair)//2 + 10]]).T
    
    if flair_img.shape[-1] < 20:
        n_zero = 20 - flair_img.shape[-1]
        flair_img = np.concatenate((flair_img, np.zeros((img_size, img_size, n_zero))), axis = -1)
    #print(flair_img.shape)'''
   
           
    '''
    #print(t2w_img.shape)
    t1w_img = np.array([get_image(a) for a in t1w[len(t1w)//2 - 8:len(t1w)//2 + 8]]).T
    if t1w_img.shape[-1] < 16:
        n_zero = 20- t1w_img.shape[-1]
        t1w_img = np.concatenate((t1w_img, np.zeros((img_size, img_size, n_zero))), axis = -1)
    #print(t1w_img.shape)

    
    t1wce_img = np.array([get_image(a) for a in t1wce[len(t1wce)//2 - 8:len(t1wce)//2 + 8]]).T
    if t1wce_img.shape[-1] < 16:
        n_zero = 20 - t1wce_img.shape[-1]
        t1wce_img = np.concatenate((t1wce_img, np.zeros((img_size, img_size, n_zero))), axis = -1)
    #print(t1wce_img.shape)
        '''
   
    t2w_img = np.array([get_image(a) for a in t2w[len(t2w)//2 - 8:len(t2w)//2 + 8]]).T
    if t2w_img.shape[-1] < 16:
        n_zero = 16 - t2w_img.shape[-1]
        t2w_img = np.concatenate((t2w_img, np.zeros((img_size, img_size, n_zero))), axis = -1)
    #return np.concatenate((flair_img, t1w_img, t1wce_img, t2w_img), axis = -1)
    return t2w_img
            #t1w_img, t1wce_img, t2w_img)
    #return flair_img

All images for each category are stored as multidimetional array

In [ ]:
flair_img=[]
flair_img_label=[]
t1w_img=[]
t1w_img_label=[]
t1wce_img=[]
t1wce_img_label=[]
t2w_img=[]
t2w_img_label=[]
print(df_train.iloc[0])
#
for i in range(500):
    images=load_3d_dicom_images(df_train.iloc[i])
    flair_img.append(images)
    flair_img_label.append(df_train.iloc[i][1])
    '''t1w_img.append(images[0])
    t1w_img_label.append(df_train.iloc[i][1])
    t1wce_img.append(images[0])
    t1wce_img_label.append(df_train.iloc[i][1])
    t2w_img.append(images[0])
    t2w_img_label.append(df_train.iloc[i][1])'''


Fitting MRI data to efficentnet 

In [ ]:
trainig_data=np.asarray(flair_img[:450])

trainig_labels=flair_img_label[:450]

val_data=np.asarray(flair_img[451:500])

val_labels=flair_img_label[451:500]

In [ ]:
#train_dataset = tf.data.Dataset.from_tensor_slices((trainig_data, trainig_labels))

#val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels))

In [ ]:

training_data=tf.concat(trainig_data,0)

label=tf.convert_to_tensor(trainig_labels, dtype=tf.float32)

val_data1=tf.concat(val_data,0)

label1=tf.convert_to_tensor(val_labels, dtype=tf.float32)

In [ ]:
#print(train_dataset.output_types)

    Convering the numpy array to tensor format

In [ ]:

import os
import glob
from tqdm import tqdm_notebook as tqdm
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import transforms, utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import tensorflow as tf

In [ ]:
def MRIModel():
    base_model = tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet')
    base_model.trainabe = False

    inputs = Input((256, 256, 16))
    x = Conv2D(3, kernel_size=(3, 3), padding='same', activation='relu')(inputs)
    x = base_model(x, training=False)
    flattened_output = GlobalAveragePooling2D()(x)
    outputs = Dense(1, activation='sigmoid', name="lstm_sigmoid")(flattened_output)
    return Model(inputs, outputs)

tf.keras.backend.clear_session()
model = MRIModel()
model.summary()

In [ ]:
# Callbacks
earlystopper = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, verbose=0, mode='min',
    restore_best_weights=True
)

In [ ]:
tf.keras.backend.clear_session() 
model = MRIModel()
model.compile('adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.AUC()])



checkpoint_filepath = 'best_model.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
filepath=checkpoint_filepath,
save_weights_only=False,
monitor='val_auc',
mode='max',
save_best_only=True,
save_freq='epoch')





# Train
history = model.fit(training_data,label, 
              epochs=50,
              validation_data=(val_data1,label1),
              callbacks=[model_checkpoint_callback])

# Evaluate
#loss, acc = model.evaluate((flair_img[:10],flair_img_label[:10]))
#wandb.log({'Val Accuracy': round(acc, 3)})



In [ ]:
plt.figure()
plt.plot(history.history['loss'],'-',label='loss')
plt.plot(history.history['val_loss'],'--',label='val_loss')
plt.legend()
plt.xlabel('Epoch #')
plt.ylabel('Loss')
plt.title(" T2W MRI Type")

plt.figure()
plt.plot(history.history['auc'],'-',label='AUC')
plt.plot(history.history['val_auc'],'--',label='val_AUC')
plt.legend()
plt.xlabel('Epoch #')
plt.ylabel('AUC')
plt.title(" T2W MRI Type")


load the images

In [ ]:
'''# let's write a simple pytorch dataloader


class BrainTumor(Dataset):
    def __init__(self, split = "train", validation_split = 0.0):
        # labels    
        self.split="train"
        if split == "valid":
            
            self.idx = df_train.iloc[:int(len(df_train.index)*validation_split)  ].index.tolist() # first 20% as validation
            self.df = df_train.loc[self.idx]
        elif split == "train":
            self.idx = df_train.iloc[int(len(df_train.index)*validation_split): ].index.tolist() # last 80% as train
            self.df=df_train.loc[self.idx]
        else:
            self.df = df_test.iloc[int(len(df_test.index)) ].index.tolist()
    def __len__(self):
        return len(self.idx )
    
    def __getitem__(self, idx):
        imgs = load_3d_dicom_images(self.df.iloc[idx])
        #imgs=np.concatenate(imgs, axis = -1)
        #transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,) * 200, (0.5,) * 200)])
        #imgs = transform(imgs)
        label = self.labels[self.ids[idx]]
        if self.split != "test":
            return torch.tensor(imgs, dtype = torch.float32))
        else:
            return torch.tensor(imgs, dtype = torch.float32)
''

In [ ]:
# testing the dataloader
train_dataset = BrainTumor()
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=8)
# val_dataset = BrainTumor(split="valid")
# val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=8)

Displaying one image

In [ ]:
fig = plt.figure(figsize=(10, 7))

  
#data = pydicom.dcmread('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T1w/Image-25.dcm')
img1 = get_image('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T1w/Image-25.dcm', True)
fig.add_subplot(2, 2, 1)
plt.imshow(img1, cmap='gray')
plt.axis('off')

https://www.kaggle.com/v1olet1nor1/rsna-custom-train

https://www.kaggle.com/trinayanbharadwaj/rsna-miccai-competition-model-building

Ref: https://www.kaggle.com/arnabs007/part-1-rsna-miccai-btrc-understanding-the-data